In [1]:
client_id = ''
client_secret = ''

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 jybcvbhl9ilclypue1xa1xsyly1tlk


In [2]:
oauth_token = ''

In [5]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/gerardromero"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {url}:"

{'data': [{'id': '605221125', 'login': 'gerardromero', 'display_name': 'gerardromero', 'type': '', 'broadcaster_type': 'partner', 'description': 'Periodista deportivo siguiendo el día a día del Barça.   Colaborador oficial de @newskill_ES Cada día directo a las 12h.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/49bdc39f-4b83-491b-8b99-8719ed8faca9-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2020-11-08T20:15:21Z'}]}


In [9]:
data = {'data': [{'id': '605221125', 'login': 'gerardromero', 'display_name': 'gerardromero', 'type': '', 'broadcaster_type': 'partner', 'description': 'Periodista deportivo siguiendo el día a día del Barça.   Colaborador oficial de @newskill_ES Cada día directo a las 12h.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/49bdc39f-4b83-491b-8b99-8719ed8faca9-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2020-11-08T20:15:21Z'}]}
data_dict = data['data'][0]
data_dict = data['data'][0]
id_canal =  list(data_dict.values())[0]
id_canal

'605221125'

In [10]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             82 non-null     int64         
 1   URL del VOD            82 non-null     object        
 2   Nombre del VOD         82 non-null     object        
 3   Duración               82 non-null     object        
 4   Fecha de inicio        82 non-null     datetime64[ns]
 5   Fecha de finalización  82 non-null     datetime64[ns]
 6   Visitas                82 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 29.0 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1892379692,https://www.twitch.tv/videos/1892379692,jijantes_fc_|actualidad_barça._carpeta_neymar....,2h1m9s,2023-08-07 09:55:15,2023-08-07 09:55:15,259
1,1891928464,https://www.twitch.tv/videos/1891928464,"jijantes_fc_|_mercato_fcb._dembelé,_kessie,_an...",3h14m40s,2023-08-06 20:01:16,2023-08-06 20:01:16,236128
2,1890683225,https://www.twitch.tv/videos/1890683225,jijantes_fc_|_actualidad_barça._mercato_del_fc...,2h24m54s,2023-08-05 09:56:15,2023-08-05 09:56:15,108138
3,1889799073,https://www.twitch.tv/videos/1889799073,jijantes_fc_|_actualidad_barça._las_salidas_li...,3h48m10s,2023-08-04 09:56:10,2023-08-04 09:56:10,184809
4,1888926577,https://www.twitch.tv/videos/1888926577,jijantes_fc_|_actualidad_barça._cumbre_deporti...,3h53m38s,2023-08-03 09:55:59,2023-08-03 09:55:59,198580


In [11]:
df.to_csv('twitch_api_gerardromero_csv.csv', index=False)